In [1]:
from bs4 import BeautifulSoup
from pgmpy.factors.discrete import TabularCPD
from pgmpy.models import BayesianNetwork
from pgmpy.inference import VariableElimination
import pandas as pd

In [2]:
class MyCPD:
    def __init__(self, name, states, probs, cpd_parents=None):
        self.name = name
        self.states = states
        self.probs = probs
        if cpd_parents:
            self.parents = cpd_parents
        else:
            self.parents = []

        for parent in self.parents:
            parent_list.append((parent, self.name))

        self.states_names = {name: states}


def get_ev_cards(parents):
    return [len(cpds_done[parent].states) for parent in parents]


def get_state_names(parents, cpd: MyCPD):
    cpd_states_names = {key: val for key, val in cpd.states_names.items()}
    for parent in parents:
        cpd_states_names.update(cpds_done[parent].states_names)
    return cpd_states_names


def parse_states(state_list):
    return [state['id'] for state in state_list]


def parse_parents(parents_list):
    return parents_list.string.split()


def parse_probabilities(states_no, probabilities):
    probabilities = probabilities.split()
    probs = [[] for _ in range(states_no)]
    i = 0
    for p in probabilities:
        probs[i].append(float(p))
        i = (i + 1) % states_no
    return probs


def parse_res_states(res_states):
    probabilities = res_states.string.replace('Nie', '0.0')
    probabilities = probabilities.replace('Tak', '1.0')
    probs = [float(p) for p in probabilities.split()]
    con_probs = [1 - p for p in probs]
    return [probs, con_probs]


def no_parents_cpd(cpd_form):
    cpd_name = cpd_form['id']
    cpd_states = parse_states(cpd_form.find_all('state'))
    cpd_probabilities = parse_probabilities(len(cpd_states), cpd_form.probabilities.string)

    cpd = MyCPD(cpd_name, cpd_states, cpd_probabilities)
    return cpd


def flower_nodes_cpd(cpd_form):
    cpd_name = cpd_form['id']
    cpd_states = parse_states(cpd_form.find_all('state'))
    cpd_parents = parse_parents(cpd_form.parentss)
    resulting_states = parse_res_states(cpd_form.resultingstates)

    cpd = MyCPD(cpd_name, cpd_states, resulting_states, cpd_parents)
    return cpd


def node_with_parents(cpd_form):
    cpd_name = cpd_form['id']
    cpd_states = parse_states(cpd_form.find_all('state'))
    cpd_probabilities = parse_probabilities(len(cpd_states), cpd_form.probabilities.string)
    cpd_parents = parse_parents(cpd_form.parentss)

    return MyCPD(cpd_name, cpd_states, cpd_probabilities, cpd_parents)


def parse_cpd(cpd_form):
    if 'deterministic' in str(cpd_form):
        return flower_nodes_cpd(cpd_form)
    elif 'parents' in str(cpd_form):
        return node_with_parents(cpd_form)
    else:
        cdp = no_parents_cpd(cpd_form)
        questions.append((cdp.name, cdp.states_names[cdp.name]))
        return cdp


In [3]:
with open('Kwiotki.txt') as f:
    text = f.read().replace('parents', 'parentss')

soup = BeautifulSoup(text, 'html.parser')

nodes = BeautifulSoup(str(soup.nodes), 'html.parser')
cpds = nodes.findAll('cpt')
cpds.extend(nodes.findAll('deterministic'))

cpds_not_done = {}
cpds_done = {}
cpds_model = []
parent_list = []
questions = []

for cpd in cpds:
    cpds_not_done[cpd['id']] = cpd

for cpd_id, cpd in cpds_not_done.items():
    cpds_done[cpd_id] = parse_cpd(cpd)

for cpd_id, cpd in cpds_done.items():
#     print(cpd_id, cpd.name, cpd.states, cpd.parents)
    if cpd.parents:
        tcpd = TabularCPD(cpd.name, len(cpd.states), cpd.probs,
                          evidence=cpd.parents, evidence_card=get_ev_cards(cpd.parents),
                          state_names=get_state_names(cpd.parents, cpd))
    else:
        tcpd = TabularCPD(cpd.name, len(cpd.states), cpd.probs, state_names=cpd.states_names)

    cpds_model.append(tcpd)

model = BayesianNetwork(parent_list)
model.add_cpds(*cpds_model)

In [4]:
model.check_model()

True

In [128]:

model.check_model()
daft = model.to_daft()
# daft.render()

In [5]:
questions

[('Roślinka_ma_być_jadalna', ['Tak', 'Nie']),
 ('Lubię_naturalną_pielęgnację_ciała', ['Tak', 'Nie']),
 ('Mam_dużo_czasu', ['Tak', 'Nie']),
 ('Lubię_babrać_się_w_ziemi', ['Tak', 'Nie']),
 ('Mam_balkon', ['Tak', 'Nie']),
 ('Kwiatek_na_podłodze', ['Tak', 'Nie']),
 ('Lubię_podlewać', ['Tak', 'Nie']),
 ('Jestem_obowiązkowy', ['Tak', 'Nie']),
 ('Mam_zwierzę_wolnochodzące', ['Tak', 'Nie']),
 ('Mam_dziecko', ['Tak', 'Nie']),
 ('Jestem_alergikiem', ['Tak', 'Nie']),
 ('W_domu_jest_wysoka_wilgotność', ['Tak', 'Nie']),
 ('Mam_nawilżacz_powietrza', ['Tak', 'Nie']),
 ('Kwiatek_na_parapecie', ['Tak', 'Nie']),
 ('Strona_okna', ['Północ', 'Południe', 'Wschód', 'Zachód']),
 ('Kwitnienie', ['Całoroczne', 'Sezonowe', 'Nigdy']),
 ('Ulubione_kolory', ['Biały', 'Zielony', 'Różowy'])]

In [130]:
print('{')
for quest in questions:
    print(f'"{quest[0]}": "",               #{quest[1]}')
print('}')

{
"Roślinka_ma_być_jadalna": "",               #['Tak', 'Nie']
"Lubię_naturalną_pielęgnację_ciała": "",               #['Tak', 'Nie']
"Mam_dużo_czasu": "",               #['Tak', 'Nie']
"Lubię_babrać_się_w_ziemi": "",               #['Tak', 'Nie']
"Mam_balkon": "",               #['Tak', 'Nie']
"Kwiatek_na_podłodze": "",               #['Tak', 'Nie']
"Lubię_podlewać": "",               #['Tak', 'Nie']
"Jestem_obowiązkowy": "",               #['Tak', 'Nie']
"Mam_zwierzę_wolnochodzące": "",               #['Tak', 'Nie']
"Mam_dziecko": "",               #['Tak', 'Nie']
"Jestem_alergikiem": "",               #['Tak', 'Nie']
"W_domu_jest_wysoka_wilgotność": "",               #['Tak', 'Nie']
"Mam_nawilżacz_powietrza": "",               #['Tak', 'Nie']
"Kwiatek_na_parapecie": "",               #['Tak', 'Nie']
"Strona_okna": "",               #['Północ', 'Południe', 'Wschód', 'Zachód']
"Kwitnienie": "",               #['Całoroczne', 'Sezonowe', 'Nigdy']
"Ulubione_kolory": "",               #['

In [6]:
d_quests = {
"Roślinka_ma_być_jadalna": ["Nie"],               #['Tak', 'Nie']
"Lubię_naturalną_pielęgnację_ciała": ["Nie"],               #['Tak', 'Nie']
"Mam_dużo_czasu": ["Nie"],               #['Tak', 'Nie']
"Lubię_babrać_się_w_ziemi": ["Tak"],               #['Tak', 'Nie']
"Mam_balkon": ["Tak"],               #['Tak', 'Nie']
"Kwiatek_na_podłodze": ["Tak"],               #['Tak', 'Nie']
"Lubię_podlewać": ["Tak"],               #['Tak', 'Nie']
"Jestem_obowiązkowy": ["Nie"],               #['Tak', 'Nie']
"Mam_zwierzę_wolnochodzące": ["Tak"],               #['Tak', 'Nie']
"Mam_dziecko": ["Tak"],               #['Tak', 'Nie']
"Jestem_alergikiem": ["Tak"],               #['Tak', 'Nie']
"W_domu_jest_wysoka_wilgotność": ["Nie"],               #['Tak', 'Nie']
"Mam_nawilżacz_powietrza": ["Nie"],               #['Tak', 'Nie']
"Kwiatek_na_parapecie": ["Nie"],               #['Tak', 'Nie']
"Strona_okna": ["Południe"],               #['Północ', 'Południe', 'Wschód', 'Zachód']
"Kwitnienie": ["Nigdy"],               #['Całoroczne', 'Sezonowe', 'Nigdy']
"Ulubione_kolory": ["Zielony"],               #['Biały', 'Zielony', 'Różowy']
"Kwiatek_na_balkonie": ["Nie"]
}

In [7]:
f_quests = {
"Roślinka_ma_być_jadalna": ["Nie"],               #['Tak', 'Nie']
"Lubię_naturalną_pielęgnację_ciała": ["Nie"],               #['Tak', 'Nie']
"Mam_dużo_czasu": ["Nie"],               #['Tak', 'Nie']
"Lubię_babrać_się_w_ziemi": ["Tak"],               #['Tak', 'Nie']
"Mam_balkon": ["Tak"],               #['Tak', 'Nie']
"Kwiatek_na_podłodze": ["Tak"],               #['Tak', 'Nie']
"Lubię_podlewać": ["Tak"],               #['Tak', 'Nie']
"Jestem_obowiązkowy": ["Nie"],               #['Tak', 'Nie']
"Mam_zwierzę_wolnochodzące": ["Nie"],               #['Tak', 'Nie']
"Mam_dziecko": ["Nie"],               #['Tak', 'Nie']
"Jestem_alergikiem": ["Nie"],               #['Tak', 'Nie']
"W_domu_jest_wysoka_wilgotność": ["Nie"],               #['Tak', 'Nie']
"Mam_nawilżacz_powietrza": ["Nie"],               #['Tak', 'Nie']
"Kwiatek_na_parapecie": ["Nie"],               #['Tak', 'Nie']
"Strona_okna": ["Południe"],               #['Północ', 'Południe', 'Wschód', 'Zachód']
"Kwitnienie": ["Nigdy"],               #['Całoroczne', 'Sezonowe', 'Nigdy']
"Ulubione_kolory": ["Zielony"],               #['Biały', 'Zielony', 'Różowy']
"Kwiatek_na_balkonie": ["Nie"]
}

In [8]:
df = pd.DataFrame.from_dict(f_quests)

In [9]:
df

,Roślinka_ma_być_jadalna,Lubię_naturalną_pielęgnację_ciała,Mam_dużo_czasu,Lubię_babrać_się_w_ziemi,Mam_balkon,Kwiatek_na_podłodze,Lubię_podlewać,Jestem_obowiązkowy,Mam_zwierzę_wolnochodzące,Mam_dziecko,Jestem_alergikiem,W_domu_jest_wysoka_wilgotność,Mam_nawilżacz_powietrza,Kwiatek_na_parapecie,Strona_okna,Kwitnienie,Ulubione_kolory,Kwiatek_na_balkonie
0,Nie,Nie,Nie,Tak,Tak,Tak,Tak,Nie,Nie,Nie,Nie,Nie,Nie,Nie,Południe,Nigdy,Zielony,Nie


In [10]:
y = model.predict(df)

In [11]:
y[['Grudnik', 'Dracena', 'Wężownica', 'Kaktus', 'Storczyk', 'Aloes', 'Monstera', 'Fiołek_afrykański', 'Mięta', 'Skrzydłokwiat', 'Paproć', 'Fikus']]

,Grudnik,Dracena,Wężownica,Kaktus,Storczyk,Aloes,Monstera,Fiołek_afrykański,Mięta,Skrzydłokwiat,Paproć,Fikus
0,Nie,Nie,Nie,Nie,Nie,Nie,Nie,Nie,Nie,Nie,Nie,Tak


In [12]:
y['Dużo_nawożenia']

0    Mało
Name: Dużo_nawożenia, dtype: object

In [13]:
df = model.simulate(1000)


In [14]:
df[df['Grudnik'] == 'Tak']

,Jestem_obowiązkowy,Mam_nawilżacz_powietrza,Duże_nawodnienie,Kaktus,Kwitnienie,Lubię_babrać_się_w_ziemi,Lubię_naturalną_pielęgnację_ciała,Kwiatek_na_podłodze,Dużo_nawożenia,Grudnik,...,Mam_dziecko,Strona_okna,W_domu_jest_wysoka_wilgotność,Mam_dużo_czasu,Kwiatek_na_parapecie,Mięta,Częste_przesadzanie,Mam_zwierzę_wolnochodzące,Użytkowa,Storczyk
224,Nie,Nie,Nie,Nie,Sezonowe,Tak,Nie,Nie,Mało,Tak,...,Nie,Wschód,Nie,Nie,Tak,Nie,Nie,Tak,Nie,Nie
785,Tak,Tak,Nie,Nie,Sezonowe,Nie,Nie,Nie,Mało,Tak,...,Tak,Zachód,Nie,Nie,Nie,Nie,Nie,Nie,Nie,Nie


In [43]:
df.loc[316]

W_domu_jest_wysoka_wilgotność             Nie
Lubię_naturalną_pielęgnację_ciała         Tak
Grudnik                                   Tak
Aloes                                     Nie
Kwiatek_na_balkonie                       Nie
Strona_okna                            Północ
Mam_nawilżacz_powietrza                   Nie
Kwitnienie                           Sezonowe
Roślinka_ma_być_jadalna                   Nie
wężownica                                 Nie
Kwiatek_na_podłodze                       Tak
Kwiatek_na_parapecie                      Nie
Dużo_nawożenia                           Mało
Paproć                                    Nie
skrzydłokwiat                             Nie
Ulubione_kolory                        Różowy
Duże_nasłonecznienie                      Nie
Duże_nawodnienie                          Nie
Trująca                                   Nie
Mam_dużo_czasu                            Nie
Mięta                                     Nie
Dracena                           

In [15]:
infer = VariableElimination(model)

In [ ]:
# posterior_p = infer.query(variables=['Book'], evidence={'Genre': 'western', 'Language': 'polish', 'Length': 'Long', 'Format': 'paperback'})

In [ ]:
# independencies = model.get_independencies()

In [ ]:
# independencies

In [23]:
quests = {key: val[0] for key, val in f_quests.items()}

In [26]:
infer = VariableElimination(model)
posterior_p = infer.query(['Wężownica'], evidence=quests)
print(posterior_p)
posterior_p = infer.query(['Kaktus'], evidence=quests)
print(posterior_p)
posterior_p = infer.query(['Grudnik'], evidence=quests)
print(posterior_p)
posterior_p = infer.query(['Storczyk'], evidence=quests)
print(posterior_p)
posterior_p = infer.query(['Aloes'], evidence=quests)
print(posterior_p)
posterior_p = infer.query(['Mięta'], evidence=quests)
print(posterior_p)
posterior_p = infer.query(['Skrzydłokwiat'], evidence=quests)
print(posterior_p)
posterior_p = infer.query(['Fikus'], evidence=quests)
print(posterior_p)
posterior_p = infer.query(['Fiołek_afrykański'], evidence=quests)
print(posterior_p)
posterior_p = infer.query(['Dracena'], evidence=quests)
print(posterior_p)
posterior_p = infer.query(['Paproć'], evidence=quests)
print(posterior_p)
posterior_p = infer.query(['Monstera'], evidence=quests)
print(posterior_p)


+----------------+------------------+
| Wężownica      |   phi(Wężownica) |
+================+==================+
| Wężownica(Tak) |           0.0000 |
+----------------+------------------+
| Wężownica(Nie) |           1.0000 |
+----------------+------------------+




+-------------+---------------+
| Kaktus      |   phi(Kaktus) |
+=============+===============+
| Kaktus(Tak) |        0.0000 |
+-------------+---------------+
| Kaktus(Nie) |        1.0000 |
+-------------+---------------+




+--------------+----------------+
| Grudnik      |   phi(Grudnik) |
+==============+================+
| Grudnik(Tak) |         0.0000 |
+--------------+----------------+
| Grudnik(Nie) |         1.0000 |
+--------------+----------------+




+---------------+-----------------+
| Storczyk      |   phi(Storczyk) |
+===============+=================+
| Storczyk(Tak) |          0.0000 |
+---------------+-----------------+
| Storczyk(Nie) |          1.0000 |
+---------------+-----------------+




+------------+--------------+
| Aloes      |   phi(Aloes) |
+============+==============+
| Aloes(Tak) |       0.0000 |
+------------+--------------+
| Aloes(Nie) |       1.0000 |
+------------+--------------+



...


+------------+--------------+
| Fikus      |   phi(Fikus) |
+============+==============+
| Fikus(Tak) |       0.0998 |
+------------+--------------+
| Fikus(Nie) |       0.9002 |
+------------+--------------+




+------------------------+--------------------------+
| Fiołek_afrykański      |   phi(Fiołek_afrykański) |
+========================+==========================+
| Fiołek_afrykański(Tak) |                   0.0000 |
+------------------------+--------------------------+
| Fiołek_afrykański(Nie) |                   1.0000 |
+------------------------+--------------------------+




+--------------+----------------+
| Dracena      |   phi(Dracena) |
+==============+================+
| Dracena(Tak) |         0.0000 |
+--------------+----------------+
| Dracena(Nie) |         1.0000 |
+--------------+----------------+




+-------------+---------------+
| Paproć      |   phi(Paproć) |
+=============+===============+
| Paproć(Tak) |        0.0000 |
+-------------+---------------+
| Paproć(Nie) |        1.0000 |
+-------------+---------------+




+---------------+-----------------+
| Monstera      |   phi(Monstera) |
+===============+=================+
| Monstera(Tak) |          0.0000 |
+---------------+-----------------+
| Monstera(Nie) |          1.0000 |
+---------------+-----------------+
